In [107]:
import pandas as pd
import os
import glob
import numpy as np
from sklearn.preprocessing import MinMaxScaler

hide_toggle()

In [106]:
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

hide_toggle()

In [84]:
path=r'C:\Users\stang\Google Drive\Data_for_fun\Fantasy Basketball\DailyGameLogs'
allFiles = glob.glob(path + "/*.csv")

gamelogs = pd.concat([pd.read_csv(f, encoding='cp1252') for f in glob.glob('*.csv')])

hide_toggle()

In [85]:
path1=r'C:\Users\stang\Google Drive\Data_for_fun\Fantasy Basketball'

gamelogs.to_csv(os.path.join(path1,r'GameLogs.csv'))

hide_toggle()

In [86]:
gamelogs['Date'] = pd.to_datetime(gamelogs['Date'])
gamelogs.sort_values(by=['Player','Date'], inplace=True, ascending=False) 
# Want latest game to be game 1 - easy filter for last 10 games
gamelogs['Game_Num'] = gamelogs.groupby('Player').cumcount() + 1

hide_toggle()

### Full Season

In [87]:
data_group = gamelogs.groupby(by=["Player"]).agg({'GmSc':'size','PTS':['sum'],'TRB':['sum'],'BLK':['sum'], 'STL':['sum'], \
                                              'AST':['sum'], '3P':['sum'], 'FG':['sum'], 'FGA':['sum'], 'FT':['sum'], \
                                              'FTA':['sum'], 'TOV':['sum']})

hide_toggle()

In [88]:
data_group["PTS_G"] = data_group["PTS", "sum"] / data_group["GmSc", "size"]
data_group["TRB_G"] = data_group["TRB", "sum"] / data_group["GmSc", "size"]
data_group["BLK_G"] = data_group["BLK", "sum"] / data_group["GmSc", "size"]
data_group["STL_G"] = data_group["STL", "sum"] / data_group["GmSc", "size"]
data_group["AST_G"] = data_group["AST", "sum"] / data_group["GmSc", "size"]
data_group["3P_G"] = data_group["3P", "sum"] / data_group["GmSc", "size"]

data_group["FGA"] = np.where(data_group["FGA"] == 0, 1, data_group["FGA"])
data_group["FG%"] = data_group["FG", "sum"]**2 / data_group["GmSc", "size"]
# data_group["FGA", "sum"]

data_group["FTA"] = np.where(data_group["FTA"] == 0, 1, data_group["FTA"])
data_group["FT%"] = data_group["FT", "sum"]**2 / data_group["FTA", "sum"] 
# data_group["FTA", "sum"]

data_group["TOV_G"] = data_group["TOV", "sum"] / data_group["GmSc", "size"]

hide_toggle()

In [89]:
column_names_to_normalize = ['PTS_G', 'TRB_G', 'BLK_G', 'STL_G', 'AST_G', '3P_G', 'FG%', 'FT%', 'TOV_G']
x = data_group[column_names_to_normalize].values
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = data_group.index)
data_group[column_names_to_normalize] = df_temp

hide_toggle()

In [90]:
data_group["sum"] = data_group["PTS_G"] + data_group["TRB_G"] + data_group["BLK_G"] + data_group["STL_G"] + data_group["AST_G"] + \
                    data_group["3P_G"] + data_group["FG%"] + data_group["FT%"] - data_group["TOV_G"] 
data_group = data_group.sort_values(by=['sum'], ascending=False)

hide_toggle()

In [91]:
data_group1 = data_group.loc[:, ['PTS_G', 'TRB_G', 'BLK_G', 'STL_G', 'AST_G', '3P_G', 'FG%', 'FT%', 'TOV_G', 'sum']]

hide_toggle()

In [92]:
team = pd.read_csv(r'C:\Users\stang\Google Drive\Data_for_fun\Fantasy Basketball\FBteam.csv', encoding='cp1252')

hide_toggle()

In [93]:
team_stats = data_group1[data_group1.index.isin(team["Player"])]
total = team_stats.sum(axis=0)

hide_toggle()

In [96]:
team_stats

,PTS_G,TRB_G,BLK_G,STL_G,AST_G,3P_G,FG%,FT%,TOV_G,sum
,,,,,,,,,,
Player,,,,,,,,,,
Paul George,0.804924,0.522505,0.248067,0.853365,0.420906,0.715000,0.738984,0.542683,0.548571,4.297863
LeBron James,0.831956,0.523750,0.251484,0.512821,0.687104,0.422303,0.927131,0.492451,0.615065,4.033934
Kemba Walker,0.795900,0.274433,0.180412,0.531674,0.584245,0.734118,0.764046,0.551187,0.451765,3.964252
Nikola Vucevic,0.619835,0.757161,0.382693,0.384615,0.358233,0.245818,0.658709,0.259005,0.376797,3.289273
Marc Gasol,0.492195,0.540829,0.535770,0.524476,0.443387,0.321455,0.299089,0.344369,0.387879,3.113691
Victor Oladipo,0.591515,0.398278,0.144330,0.646154,0.523256,0.449280,0.353915,0.232378,0.446171,2.892934
Robert Covington,0.396780,0.354207,0.417204,0.829327,0.121124,0.494000,0.171043,0.205326,0.245714,2.743297
Nikola Mirotic,0.527778,0.581866,0.240550,0.288462,0.100937,0.520000,0.248024,0.225347,0.213333,2.519629


In [43]:
total.sort_values()

FT%         3.990259
BLK_G       4.463116
TOV_G       5.060253
AST_G       5.244171
FG%         5.603810
3P_G        5.819475
TRB_G       6.075209
PTS_G       7.601643
STL_G       7.752249
sum        41.489679
dtype: float64

### Last 10 Games

In [97]:
data_group_10 = gamelogs[gamelogs['Game_Num'] <= 10]
data_group_10 = data_group_10.groupby(by=["Player"]).agg({'GmSc':'size','PTS':['sum'],'TRB':['sum'],'BLK':['sum'], \
                                                    'STL':['sum'], 'AST':['sum'], '3P':['sum'], 'FG':['sum'], 'FGA':['sum'], \
                                                    'FT':['sum'], 'FTA':['sum'], 'TOV':['sum']})

hide_toggle()

In [98]:
data_group_10["PTS_G"] = data_group_10["PTS", "sum"] / data_group_10["GmSc", "size"]
data_group_10["TRB_G"] = data_group_10["TRB", "sum"] / data_group_10["GmSc", "size"]
data_group_10["BLK_G"] = data_group_10["BLK", "sum"] / data_group_10["GmSc", "size"]
data_group_10["STL_G"] = data_group_10["STL", "sum"] / data_group_10["GmSc", "size"]
data_group_10["AST_G"] = data_group_10["AST", "sum"] / data_group_10["GmSc", "size"]
data_group_10["3P_G"] = data_group_10["3P", "sum"] / data_group_10["GmSc", "size"]

data_group_10["FGA"] = np.where(data_group_10["FGA"] == 0, 1, data_group_10["FGA"])
data_group_10["FG%"] = data_group_10["FG", "sum"]**2 / data_group_10["GmSc", "size"]
# data_group["FGA", "sum"]

data_group_10["FTA"] = np.where(data_group_10["FTA"] == 0, 1, data_group_10["FTA"])
data_group_10["FT%"] = data_group_10["FT", "sum"]**2 / data_group_10["FTA", "sum"] 
# data_group["FTA", "sum"]

data_group_10["TOV_G"] = data_group_10["TOV", "sum"] / data_group_10["GmSc", "size"]

hide_toggle()

In [99]:
x_10 = data_group_10[column_names_to_normalize].values
x_scaled_10 = scaler.fit_transform(x_10)
df_temp_10 = pd.DataFrame(x_scaled_10, columns=column_names_to_normalize, index = data_group_10.index)
data_group_10[column_names_to_normalize] = df_temp_10

hide_toggle()

In [100]:
data_group_10["sum"] = data_group_10["PTS_G"] + data_group_10["TRB_G"] + data_group_10["BLK_G"] + data_group_10["STL_G"] + \
                        data_group_10["AST_G"] + data_group_10["3P_G"] + data_group_10["FG%"] + data_group_10["FT%"] - \
                        data_group_10["TOV_G"] 
data_group_10 = data_group_10.sort_values(by=['sum'], ascending=False)

hide_toggle()

In [101]:
data_group_10 = data_group_10.loc[:, ['PTS_G', 'TRB_G', 'BLK_G', 'STL_G', 'AST_G', '3P_G', 'FG%', 'FT%', 'TOV_G', 'sum']]

hide_toggle()

In [103]:
team_stats_10 = data_group_10[data_group_10.index.isin(team["Player"])]
total_10 = team_stats_10.sum(axis=0)

hide_toggle()

In [104]:
team_stats_10

,PTS_G,TRB_G,BLK_G,STL_G,AST_G,3P_G,FG%,FT%,TOV_G,sum
,,,,,,,,,,
Player,,,,,,,,,,
Paul George,0.784264,0.588235,0.096774,0.735294,0.445545,0.690909,0.848338,0.518508,0.60,4.107867
Kemba Walker,0.659898,0.287582,0.258065,0.470588,0.544554,0.727273,0.679902,0.256112,0.42,3.463973
LeBron James,0.624365,0.653595,0.096774,0.382353,0.831683,0.290909,0.637196,0.288126,0.62,3.185001
Nikola Vucevic,0.489848,0.908497,0.322581,0.264706,0.336634,0.218182,0.504848,0.135434,0.38,2.800728
Marc Gasol,0.355330,0.450980,0.483871,0.352941,0.584158,0.218182,0.192367,0.246688,0.56,2.324518
Trevor Ariza,0.352792,0.398693,0.000000,0.705882,0.425743,0.454545,0.177285,0.166336,0.40,2.281276
Robert Covington,0.360406,0.379085,0.322581,0.500000,0.168317,0.400000,0.177285,0.170256,0.28,2.197930
Victor Oladipo,0.373096,0.359477,0.064516,0.500000,0.603960,0.290909,0.184749,0.247032,0.52,2.103740


In [105]:
total_10.sort_values()

FT%         3.011900
BLK_G       3.451613
FG%         4.736688
3P_G        4.818182
TOV_G       5.660000
AST_G       5.663366
STL_G       6.058824
PTS_G       6.210660
TRB_G       6.241830
sum        34.533063
dtype: float64

### Available Players

In [109]:
rostered = pd.read_csv(r'C:\Users\stang\Google Drive\Data_for_fun\Fantasy Basketball\Rostered_Players.csv', encoding='cp1252')

hide_toggle()

In [117]:
available = data_group1[~data_group1.index.isin(rostered["Player"])]

hide_toggle()

In [116]:
available.sort_values(by='sum', ascending=False).head(25)

# Gary Harris (injured) - returns tomorrow
# Larry Nance
# Kent Bazemore (out two weeks)
# JaVale McGee (injured)
# Otto Porter (injured)
# Brook Lopez
# Dennis Schroder
# Nemanja Bjelica

,PTS_G,TRB_G,BLK_G,STL_G,AST_G,3P_G,FG%,FT%,TOV_G,sum
,,,,,,,,,,
Player,,,,,,,,,,
John Wall,0.639205,0.236791,0.304446,0.600962,0.826672,0.344500,0.545417,0.345619,0.680000,3.163610
Tim Hardaway,0.629234,0.213653,0.053062,0.395928,0.299248,0.562824,0.419505,0.561548,0.392605,2.742395
Kent Bazemore,0.429590,0.252331,0.339600,0.678733,0.259348,0.324235,0.248667,0.250597,0.376471,2.406631
Brook Lopez,0.370370,0.259187,0.691581,0.192308,0.104974,0.525778,0.186363,0.126860,0.193016,2.264404
J.J. Redick,0.547186,0.148504,0.082474,0.087912,0.287929,0.588343,0.368814,0.380329,0.245551,2.245941
Gary Harris,0.524242,0.206654,0.162371,0.480769,0.290698,0.374400,0.232353,0.170014,0.228571,2.212929
Jaren Jackson,0.395722,0.296535,0.604912,0.361991,0.125399,0.171294,0.237366,0.231429,0.290420,2.134227
Dennis Schroder,0.487603,0.242899,0.065604,0.407925,0.446324,0.283636,0.345293,0.286251,0.515325,2.050211


In [121]:
available_10 = data_group_10[~data_group_10.index.isin(rostered["Player"])]

hide_toggle()

In [122]:
available_10.sort_values(by='sum', ascending=False).head(25)

,PTS_G,TRB_G,BLK_G,STL_G,AST_G,3P_G,FG%,FT%,TOV_G,sum
,,,,,,,,,,
Player,,,,,,,,,,
Larry Nance,0.304569,0.686275,0.322581,0.705882,0.485149,0.072727,0.169975,0.168609,0.34,2.575766
John Wall,0.540609,0.281046,0.129032,0.323529,0.940594,0.327273,0.517390,0.223187,0.80,2.482660
Thaddeus Young,0.426396,0.464052,0.258065,0.500000,0.237624,0.109091,0.377039,0.147533,0.18,2.339800
Kent Bazemore,0.431472,0.274510,0.258065,0.529412,0.386139,0.418182,0.286319,0.169345,0.48,2.273442
Brook Lopez,0.335025,0.235294,0.741935,0.205882,0.108911,0.454545,0.162819,0.121928,0.22,2.146341
Tristan Thompson,0.350254,0.869281,0.129032,0.264706,0.267327,0.000000,0.241305,0.173523,0.24,2.055428
Marvin Williams,0.317259,0.424837,0.258065,0.264706,0.118812,0.454545,0.148969,0.100329,0.08,2.007522
Otto Porter,0.317259,0.529412,0.064516,0.500000,0.247525,0.218182,0.192367,0.096969,0.16,2.006229


In [81]:
data_group1[data_group1.index == "Wendell Carter"]

,PTS_G,TRB_G,BLK_G,STL_G,AST_G,3P_G,FG%,FT%,TOV_G,sum
,,,,,,,,,,
Player,,,,,,,,,,
Wendell Carter,0.31734,0.429659,0.531214,0.181624,0.18034,0.028889,0.174534,0.223014,0.279365,1.787248


In [82]:
data_group_10[data_group_10.index == "Wendell Carter"]

,PTS_G,TRB_G,BLK_G,STL_G,AST_G,3P_G,FG%,FT%,TOV_G,sum
,,,,,,,,,,
Player,,,,,,,,,,
Wendell Carter,0.220812,0.411765,0.387097,0.088235,0.108911,0.0,0.117036,0.052673,0.26,1.126529
